In [1]:
import boto3
import pandas as pd
from botocore.config import Config


config = Config(
    region_name="us-east-1",
    signature_version='v4',
    retries={
        'max_attempts': 10,
        'mode': 'standard'
    },
    read_timeout=600
)

In [7]:
bedrock = None
import json

import logging

def get_bedrock_client():
    global bedrock
    if not bedrock:
        bedrock = boto3.client(service_name='bedrock-runtime', config=config)
    return bedrock


def invoke_llama_70b(model_id, system_prompt, user_prompt, max_tokens=2048, with_response_stream=False):
    """
    Invoke LLama-70B model
    :param model_id:
    :param system_prompt:
    :param messages:
    :param max_tokens:
    :param with_response_stream:
    :return:
    """
    try:
        llama3_prompt = """
        <|begin_of_text|><|start_header_id|>system<|end_header_id|>

        {system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

        {user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
        """
        body = {
            "prompt": llama3_prompt.format(system_prompt=system_prompt, user_prompt=user_prompt),
            "max_gen_len": 2048,
            "temperature": 0.01,
            "top_p": 0.9
        }
        if with_response_stream:
            response = get_bedrock_client().invoke_model_with_response_stream(body=json.dumps(body), modelId=model_id)
            return response
        else:
            response = get_bedrock_client().invoke_model(
                modelId=model_id, body=json.dumps(body)
            )
            response_body = json.loads(response["body"].read())
            return response_body
    except Exception as e:
        logging.error("Couldn't invoke LLama 70B")
        logging.error(e)

In [9]:
res = invoke_llama_70b("meta.llama3-70b-instruct-v1:0", "You are a friendly conversation assistant", "who are you")

In [10]:
res

{'generation': ' Hi there! I\'m so happy to meet you! I\'m a friendly conversation assistant, which means I\'m a computer program designed to chat with you in a way that feels natural and fun. I don\'t have a personal name, but you can call me "Assistant" or "Chatbot" if you like. My purpose is to help answer your questions, provide information, and even just have a nice conversation with you. I\'m constantly learning and improving, so I can become a better conversationalist over time. What would you like to talk about?',
 'prompt_token_count': 29,
 'generation_token_count': 114,
 'stop_reason': 'stop'}